In [3]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d60b18ff-3eb1-404f-9340-fb795d95eca7;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 144ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/27 17:43:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/27 17:43:59 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
from graphframes import GraphFrame

In [ ]:
# df = readCheckpoint(size='medium').cache()

In [ ]:
# df.printSchema()

In [ ]:
#filter to 3 malicious hosts

In [ ]:
# df_first_events = get_firsts(df).cache()

In [ ]:
# df_first_events.count()

In [ ]:
# x is the number of hours
# y is the maximum number of events for any hour in a graph
# x_axis = 2*(x-1)+2
# y_axis = 2*(y-1)+2

In [ ]:
# df_first_events.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/first_events")

In [5]:
window = Window.partitionBy("relationship").orderBy("timestamp")
df = readFirstEvents().withColumn("relationship", sort_array(array("actorID", "objectID")) )  \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1) \
                .drop('rank').cache()

 5:44PM UTC on Mar 27, 2023 --- read time: 6.246866226196289 seconds ---


In [6]:
df.count()

4539381

In [7]:
df.limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,...,file_path,direction,logon_id,requesting_domain,requesting_user,event_minute,event_day,event_hour,malicious,relationship
0,7d8ab102-1142-4fa4-a628-1323bbf0f212,2019-09-24 14:24:25.177,0000e8e2-c112-4610-9ddf-00b24a3fbdc5,73d89b62-3424-41ea-9cb9-f2219035997d,FILE,MODIFY,SysClient0419.systemia.com,None,None,System,...,\Device\HarddiskVolume1\Users\khendricks\AppDa...,None,None,None,None,24,24,14,0,"[0000e8e2-c112-4610-9ddf-00b24a3fbdc5, 73d89b6..."
1,feee1ea1-422a-474e-93b5-7ddc7edfe7ce,2019-09-25 13:24:00.114,00010fc7-3f0a-4d7f-a7b8-6f98d9a579d7,dd0741ca-63be-4e12-b5de-ad2818c2237d,PROCESS,CREATE,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,...,None,None,None,None,None,24,25,13,0,"[00010fc7-3f0a-4d7f-a7b8-6f98d9a579d7, dd0741c..."
2,4e5de99a-9dc4-4118-b646-b38764cdb0c0,2019-09-25 13:24:00.114,00010fc7-3f0a-4d7f-a7b8-6f98d9a579d7,dd0741ca-63be-4e12-b5de-ad2818c2237d,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\SYSTEM32\cmd.exe,...,None,None,None,None,None,24,25,13,0,"[00010fc7-3f0a-4d7f-a7b8-6f98d9a579d7, dd0741c..."
3,c8b0d80b-cbb9-4c73-a991-2d670db86ead,2019-09-25 13:24:00.114,00010fc7-3f0a-4d7f-a7b8-6f98d9a579d7,dd0741ca-63be-4e12-b5de-ad2818c2237d,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\SYSTEM32\cmd.exe,...,None,None,None,None,None,24,25,13,0,"[00010fc7-3f0a-4d7f-a7b8-6f98d9a579d7, dd0741c..."
4,d9822c16-e0a4-44ce-86e5-75bbc8fcd17b,2019-09-25 10:33:00.358,0004bb55-8f31-4c03-8cc7-6a83e7e4dcf7,792078e4-350a-49ed-bae0-4d1ee73c80f5,FILE,CREATE,SysClient0255.systemia.com,None,None,\Device\HarddiskVolume1\Python27\python.EXE,...,\Device\HarddiskVolume1\Windows\SysWOW64\confi...,None,None,None,None,33,25,10,0,"[0004bb55-8f31-4c03-8cc7-6a83e7e4dcf7, 792078e..."


In [8]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- relationship: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [9]:
print("image path: "+str(df.select("image_path").distinct().count()))
print("image path: "+str(df.select("parent_image_path").distinct().count()))

image path: 252


image path: 224


In [10]:
df = df.limit(100000).cache()

In [11]:
df.count()

100000

In [12]:
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df.selectExpr('actorID as id').distinct()
dst_vertices = df.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [13]:
# display the vertices
print("Vertices:")
g.vertices.show()

Vertices:
+--------------------+
|                  id|
+--------------------+
|74fa9c45-fb97-447...|
|76269410-b376-416...|
|0844e602-c28c-4ff...|
|47be1d43-98e3-4d1...|
|08ca327d-85f6-4fe...|
|0a1c3d68-f678-46e...|
|0b840b3b-336e-460...|
|0bd4f4f1-7c85-4d5...|
|f7daa5be-d3e3-4c1...|
|b3c4e673-7662-456...|
|1134edf6-e2a4-41a...|
|12c5e76d-e26e-4eb...|
|8bd099cf-5bcc-437...|
|1604c531-2dfa-43f...|
|4ef22254-0006-437...|
|2baf6d0b-6e83-486...|
|25b541ee-4ae7-446...|
|99b2b14f-e990-4ee...|
|28731de7-1db6-492...|
|29289b94-46d8-49f...|
+--------------------+
only showing top 20 rows



In [14]:
# display the edges
print("Edges:")
g.edges.show(5)

Edges:
+--------------------+--------------------+--------------------+-------+------+--------------------+---------+----------+--------------------+--------------------+--------+--------------------+---------+--------+-----------------+---------------+---------+
|                 src|                 dst|           timestamp| object|action|            hostname|user_name|privileges|          image_path|   parent_image_path|new_path|           file_path|direction|logon_id|requesting_domain|requesting_user|malicious|
+--------------------+--------------------+--------------------+-------+------+--------------------+---------+----------+--------------------+--------------------+--------+--------------------+---------+--------+-----------------+---------------+---------+
|73d89b62-3424-41e...|0000e8e2-c112-461...|2019-09-24 14:24:...|   FILE|MODIFY|SysClient0419.sys...|     null|      null|              System|                null|    null|\Device\HarddiskV...|     null|    null|          

In [15]:
g.inDegrees.show(5)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|0000e8e2-c112-461...|       1|
|00010fc7-3f0a-4d7...|       3|
|0004bb55-8f31-4c0...|       1|
|1ab5790a-9ca1-4e4...|       1|
|c5e98af6-4bbe-4b3...|       1|
+--------------------+--------+
only showing top 5 rows



In [16]:
# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)")
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

In [17]:
motifs2.count()

63245

In [18]:
motifs6.count()

1322

In [9]:
# filter paths to only those where all edges are connected
connected_paths = motifs.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")



In [ ]:
# get the 6 edges in each path and convert to a pandas dataframe
connected_paths.select("e1").limit(1).collect()

In [11]:
type(connected_paths)

pyspark.sql.dataframe.DataFrame

In [12]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")

In [14]:
connected_paths = spark.read.parquet(f"{s3_url_trusted}/prod/graph/test").cache()

In [15]:
connected_paths.limit(5).toPandas()

,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,"(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)"
1,"(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)"
2,"(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)"
3,"(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)"
4,"(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)"


In [20]:
connected_paths.count()

273

In [18]:
# using split and applying in ml with pypark
train, test, val = connected_paths.randomSplit([0.80, 0.15, 0.05], seed=12345)

In [22]:
val.count()

9

In [ ]:
# create new dataframe to union and create duplicate rows
df_dup = df.withColumn('id', lit(None).cast(StringType()))

In [ ]:
df_dup.limit(5).toPandas()

In [ ]:
df_new = df.union(df_dup).sort("timestamp").cache()
df.unpersist()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
# len(df_new.select('relationship').distinct().collect())

In [ ]:
tot = df_new.count()

In [ ]:
df_ratios = df_new.groupBy("object") \
              .count() \
              .withColumnRenamed('count', 'cnt_per_group') \
              .withColumn('perc_of_count_total', (col('cnt_per_group') / tot) * 100 ) \
              .cache()

In [ ]:
df_ratios.limit(5).toPandas()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")